In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, f1_score, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc,roc_auc_score
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from pprint import pprint
import scipy as sp
from sklearn.naive_bayes import GaussianNB
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report, accuracy_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier  # Example classifier
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
#from imblearn.over_sampling import SMOTE
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2
from nltk.tokenize import RegexpTokenizer


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
train_data = pd.read_csv('/content/gdrive/MyDrive/CS274/CS274P01PT2/train.csv').fillna(str(0))
test_data = pd.read_csv('/content/gdrive/MyDrive/CS274/CS274P01PT2/test.csv').fillna(str(0))

In [ ]:
for v in train_data['vote']:
  if v == None:
    v = str(0)
  else:
    v = str(v)



X_train = (train_data['category']+', ' +train_data['category']+', '+train_data['summary']  + ', '+ train_data['reviewText']+', '+str(train_data['vote'])).fillna('')#constructs train and test data
X_test = (test_data['category'] + ', '+ test_data['category'] + ', '+ test_data['summary']  + ', ' +''+test_data['reviewText']+', '+str(test_data['vote'])).fillna('')
y_train = train_data['overall'].astype(int)
y_test = []
#print(X_train.shape)

In [ ]:
exc_dict = {             #causes overfit, dont run this block!!!!!
    "1star": "One Star",
    "2star": "Two Stars",
    "3star": "Three Stars",
    "4star": "Four Stars",
    "5star": "Five Stars",
    "negatve":"bad|upset|wrong|annoy|disappointed|don't|not|waste|error|junk|terrible",
    "positive": "fast|enjoy|pleased|amazing|fantastic|recoommend|good|happy|great|genuine",
    "neutral": "ok|fine"
}#the dictionary for some keywords extraction

extrains = []#extracts keywords from review texts
extests = []
for x in train_data["reviewText"]:
  extraction_train = ""

  for feature, w in exc_dict.items():
    #print(x["summary":])
    if w in x:
      extraction_train+=feature
      extraction_train+=" "

    #print(extraction_train)
  extrains.append(extraction_train)
  #X_train = X_train + ", " + extraction_train
for x in test_data["reviewText"]:
  extraction_test = ""
  for feature, w in exc_dict.items():
    if w in x:
      extraction_test+=feature
      extraction_test += " "
  extests.append(extraction_test)
  #X_test = X_test + ", " + extraction_test
i = 0#add the result to train and test data
for er in extrains:
  X_train[i]+= er
  i+=1
j = 0
for er in extests:
  X_test[j]+= er
  j+=1

extrainss = []#extracts keywords from summary
extestss = []
for x in train_data["summary"]:
  extraction_train = ""

  for feature, w in exc_dict.items():
    #print(x["summary":])
    if w in x:
      extraction_train+=feature
      extraction_train+=" "

    #print(extraction_train)
  extrainss.append(extraction_train)
  #X_train = X_train + ", " + extraction_train
for x in test_data["summary"]:
  extraction_test = ""
  for feature, w in exc_dict.items():
    if w in x:
      extraction_test+=feature
      extraction_test += " "
  extestss.append(extraction_test)
  #X_test = X_test + ", " + extraction_test
i = 0#add the result to train and test data
for er in extrainss:
  X_train[i]+= er
  i+=1
j = 0
for er in extestss:
  X_test[j]+= er
  j+=1

In [ ]:
from nltk.corpus.reader.ycoe import wordpunct_tokenize #does not make a huge difference to the result
#nltk.download('stopwords'), stopwords has significant negative effect, so we deleted it
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')
puncs = set(['.','$', '/'])
ps = PorterStemmer()
wnl = WordNetLemmatizer()#use stemmer and Lemmatizer to handel the train and test words, does not have significant effect
#tokens_train = wordpunct_tokenize(X_train)
for x in X_train:
  #print(x)
  words = word_tokenize(x)
  for word in words:
    word = ps.stem(word)
    word = wnl.lemmatize(word, pos="v")
    for p in puncs:
      if p in word:
        word.replace(p, '')
  #wordsFiltered = [w for w in words if w not in stopWords]
  x = words
for x in X_test:
  words = word_tokenize(x)
  for word in words:
    word = ps.stem(word)
    word = wnl.lemmatize(word, pos="v")
    for p in puncs:
      if p in word:
        word.replace(p, '')
  #wordsFiltered = [w for w in words if w not in stopWords]
  x = words






In [ ]:
selector1 = SelectKBest(chi2, k = 3000)#Vectorize the train and test set, and use selector to filtor out the most useful features
tfidf_vectorizer = TfidfVectorizer(max_features=50000, ngram_range=(1,3))
X_train = tfidf_vectorizer.fit_transform(X_train)
X_test = tfidf_vectorizer.transform(X_test)

X_train = selector1.fit_transform(X_train, y_train)
X_test = selector1.transform(X_test)
#y_train = [1 if x>3 else 0 for x in y_train]
y_test = []

In [ ]:
 #split the training datasets for testing
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
X_test, y_test = X_val, y_val

In [ ]:
oversample = SMOTE()#might not be permitted, do not run this block!!!
X_train, y_train = oversample.fit_resample(X_train, y_train)

# ***MODELS***

In [ ]:

from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
best_clf = LogisticRegression( penalty='l2', random_state=23)
#y_pred = LinearSVC(random_state=0).fit(X_train, y_train).predict(X_test)
y_pred = best_clf.fit(X_train, y_train).predict(X_test)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)
auc = roc_auc_score(y_test, best_clf.predict_proba(X_test), multi_class='ovr')
macro_f1 = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)

print("Confusion Matrix:")
print(cm)
print(f"ROC AUC: {auc}")
print(f"Macro F1 Score: {macro_f1}")
print(f"Accuracy: {accuracy}\n")

Confusion Matrix:
[[816 234  55  22  28]
 [300 624 185  74  32]
 [ 91 237 583 185  58]
 [ 38  81 172 679 201]
 [ 36  37  45 181 844]]
ROC AUC: 0.883782960877786
Macro F1 Score: 0.6069554305418782
Accuracy: 0.6073997944501541



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:

best_clf = MultinomialNB(force_alpha=True)
#y_pred = LinearSVC(random_state=0).fit(X_train, y_train).predict(X_test)
y_pred = best_clf.fit(X_train, y_train).predict(X_test)

cm = confusion_matrix(y_test, y_pred)
auc = roc_auc_score(y_test, best_clf.predict_proba(X_test), multi_class='ovr')
macro_f1 = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)

print("Confusion Matrix:")
print(cm)
print(f"ROC AUC: {auc}")
print(f"Macro F1 Score: {macro_f1}")
print(f"Accuracy: {accuracy}\n")

Confusion Matrix:
[[752 324  51  14  14]
 [231 750 172  48  14]
 [ 57 313 619 128  37]
 [ 15 143 219 665 129]
 [ 19  80  72 230 742]]
ROC AUC: 0.880689119414332
Macro F1 Score: 0.610075943245367
Accuracy: 0.60431654676259



In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
best_clf = MLPClassifier(random_state=5, max_iter=5)
#y_pred = LinearSVC(random_state=0).fit(X_train, y_train).predict(X_test)
y_pred = best_clf.fit(X_train, y_train).predict(X_test)

cm = confusion_matrix(y_test, y_pred)
auc = roc_auc_score(y_test, best_clf.predict_proba(X_test), multi_class='ovr')
macro_f1 = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)

print("Confusion Matrix:")
print(cm)
print(f"ROC AUC: {auc}")
print(f"Macro F1 Score: {macro_f1}")
print(f"Accuracy: {accuracy}\n")


Confusion Matrix:
[[800 258  49  23  25]
 [250 682 193  72  18]
 [ 71 256 580 203  44]
 [ 17  70 151 753 180]
 [ 13  33  40 223 834]]
ROC AUC: 0.8961791015306968
Macro F1 Score: 0.626194499020307
Accuracy: 0.6250428228845495



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
final_classifier =  MLPClassifier(random_state=5, max_iter=4)
final_classifier.fit(X_train, y_train)
y_test_pred = final_classifier.predict(X_test)
test0 = pd.read_csv('/content/gdrive/MyDrive/CS274/CS274P01PT2/test.csv')
test_id = test0['id']
submission=pd.DataFrame()
submission['pred'] = y_test_pred
submission['id'] = test_id

print(submission.head)

<bound method NDFrame.head of       pred     id
0        1     a0
1        1     a1
2        1     a2
3        1     a3
4        2     a4
...    ...    ...
4495     5  a4495
4496     5  a4496
4497     5  a4497
4498     5  a4498
4499     5  a4499

[4500 rows x 2 columns]>


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (4) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
submission.to_csv('/content/gdrive/MyDrive/CS274/CS274P01PT3/sub_1.csv', index=False)